In [1]:
#有劃線功能 save avi#
import os
video_name = "20230405_123832_4652_A.mp4"
video_path = os.path.join("./output",video_name)
from collections import defaultdict

import cv2
import numpy as np
from ultralytics import YOLO

# 定义全局变量，用于存储鼠标点击的坐标和绘制的线段
points = []
lines = []

# 鼠标点击事件的回调函数
def mouse_callback(event, x, y, flags, param):
    global points, lines
    
    # 如果鼠标左键按下
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        
        # 当点击了两次后绘制一条线段
        if len(points) == 2:
            lines.append((points[0], points[1]))
            points = []
            
            # 当绘制了四条线段后，退出程序
            if len(lines) == 4:
                cv2.destroyAllWindows()
                
# Load the YOLOv8 model
model = YOLO("best.pt")
# model = YOLO("yolov8n.pt")
# Open the video file

cap = cv2.VideoCapture(video_path)

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Video writer
video_writer = cv2.VideoWriter("./output/OUTPUT.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
pause = True  # Flag to pause at the first frame
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        if pause:
            frame_size = frame.shape
            # Display the first frame
            cv2.namedWindow('First Frame with Drawing')
            cv2.setMouseCallback('First Frame with Drawing', mouse_callback)

            # 绘制线段
            while True:
                temp_frame = frame.copy()
                for line in lines:
                    cv2.line(temp_frame, line[0], line[1], (0, 255, 0), 2)
                
                # 在帧上实时显示绘制的线段
                cv2.imshow('First Frame with Drawing', temp_frame)
                
                key = cv2.waitKey(1) & 0xFF
                # 按下ESC键退出循环
                if key == 27 or len(lines) == 4:
                    break
            pause = False
            
        else:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, save=True, save_txt=True, persist=True, classes=[0, 1], project="./output/output")
            
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
        
            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point

                # if len(track) > 5:  # retain 90 tracks for 90 frames
                #     track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            # Display the annotated frame
            for line in lines:
                cv2.line(annotated_frame, line[0], line[1], (0, 255, 0), 2)
                
                # 在帧上实时显示绘制的线段
                cv2.imshow("YOLOv8 Tracking", annotated_frame)
            
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

            video_writer.write(annotated_frame)
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 5 no lights, 88.2ms
Speed: 4.0ms preprocess, 88.2ms inference, 1412.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to output\output\track166
1 label saved to output\output\track166\labels

0: 384x640 5 no lights, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to output\output\track167
1 label saved to output\output\track167\labels

0: 384x640 5 no lights, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to output\output\track168
1 label saved to output\output\track168\labels

0: 384x640 5 no lights, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to output\output\track169
1 label saved to output\output\track169\labels

0: 384x640 5 no lights, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results save

In [2]:
from shapely.geometry import LineString

road_edges = [LineString(line_data) for line_data in lines]

intersected_keys = []

for key, coords_list in track_history.items():
    car_direction = LineString(coords_list)

    j0 = car_direction.intersects(road_edges[0])
    j1 = car_direction.intersects(road_edges[1])
    j2 = car_direction.intersects(road_edges[2])
    j3 = car_direction.intersects(road_edges[3])

    if (j0 or j2) and (j1 or j3):
        intersected_keys.append(key)

print("Turning Vehicle ID:", intersected_keys)

import os

# 設定基目錄路徑，包含子目錄file0到file9
base_directory = './output'

# 建立一個空的列表來存放包含1的值
values_with_1 = []
unique_values = set()

def check_files_in_directory(directory, delimiter=' '):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                # 讀取txt檔案的每一行
                with open(file_path, 'r', encoding='utf-8') as file:
                    for line in file:
                        line = line.strip()
                        # 分割每一行的內容（假設以指定分隔符分隔）
                        columns = line.split(delimiter)
                        # 檢查第一欄是否包含1
                        if columns and '1' in columns[0]:
                            # 將符合條件的值加入到列表中
                            values_with_1.append((file_path, columns[-1]))
                            unique_values.add(columns[-1])
                            break  # 找到符合條件的行後跳出檢查該文件的迴圈

# 首先處理file目錄中的txt檔案及其子目錄中的檔案
file_directory = os.path.join(base_directory, './output/track/labels')
if os.path.exists(file_directory) and os.path.isdir(file_directory):
    check_files_in_directory(file_directory)

# 然後處理file2到file201目錄中的txt檔案
for i in range(2, 166):
    sub_directory = os.path.join(base_directory, f'./output/track{i}/labels')
    # 檢查子目錄是否存在
    if os.path.exists(sub_directory) and os.path.isdir(sub_directory):
        check_files_in_directory(sub_directory)

sorted_unique_values = sorted(unique_values)
int_list = [int(item) for item in sorted_unique_values]
print(f"Turning Light   ID: {int_list}")


for i in range(len(intersected_keys)):
    if intersected_keys[i] in int_list:
        print(f"id{intersected_keys[i]} 有打方向燈")
    else:
        print(f"id{intersected_keys[i]} 沒有打方向燈")

Turning Vehicle ID: [8]
Turning Light   ID: []
id8 沒有打方向燈


In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Find the minimum and maximum x, y values among all line segments
min_x = min_y = float('inf')
max_x = max_y = float('-inf')

for line in track_history.values():
    line_array = np.array(line)
    min_x = min(min_x, np.min(line_array[:, 0]))
    max_x = max(max_x, np.max(line_array[:, 0]))
    min_y = min(min_y, np.min(line_array[:, 1]))
    max_y = max(max_y, np.max(line_array[:, 1]))

# Calculate width and height of line segments
width = max_x - min_x
height = max_y - min_y

# Plot and save each line segment in a separate plot
for key, line in track_history.items():
    plt.figure()
    line_array = np.array(line)
    normalized_x = line_array[:, 0] - min_x
    normalized_y = max_y - line_array[:, 1]  # Invert y-axis
    plt.plot(normalized_x, normalized_y)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(f"Line {key}")
    plt.xlim(0, width)  # Set x-axis limits
    plt.ylim(0, height)  # Set y-axis limits
    plt.savefig(f"line_segment_{key}.png")
    plt.close()